In [1]:
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
%matplotlib inline

In [48]:
f = open('pvalues.savepkl','rb')
zstats = pickle.load(f)
pvalues = pickle.load(f)
f.close()

In [49]:
len(pvalues)

2847

In [72]:
def prune_dictionary(alpha, zstats, pvalues):
    keys_to_remove = []
    for key, val in pvalues.items():
        if pvalues[key] > alpha or zstats[key] > 0 or key == '':  # drop z>0 to keep more deaths in 2009 
            keys_to_remove.append(key)
    new_zstats = zstats.copy()
    new_pvalues = pvalues.copy()
    for key in keys_to_remove:
        del new_zstats[key]
        del new_pvalues[key]
    return new_zstats, new_pvalues

In [73]:
z, p = prune_dictionary(.00001/2846, zstats, pvalues)

In [74]:
len(z)

104